In [25]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re

In [2]:
df = pd.read_csv('../data/processed/geotagged_text_20190715.csv')

In [16]:
df = df[df['lang']=='in'].reset_index(drop=True)

In [3]:
pos = pd.read_csv('../sentiment_dicts/InSet-master/positive.tsv', sep='\t')

In [4]:
neg = pd.read_csv('../sentiment_dicts/InSet-master/negative.tsv', sep='\t')

In [30]:
wordlist = pd.concat([pos, neg], axis=0).reset_index(drop=True)
wordlist['weight'] = (wordlist['weight']+5)/10

In [32]:
word_dict.tail()

,word,weight
6604,kantong kering,0.51
6605,penggaruk,0.52
6606,senewen,0.51
6607,menetapkan,0.50
6608,kacang botor,0.52


In [35]:
word_dict = {}
for i in range(wordlist.shape[0]):
    word_dict[wordlist['word'][i]] = wordlist['weight'][i]

In [18]:
def get_words_advanced(string):
    split_words = []
    for elem in string.lower().split():
        if re.match(r'(.+)?(\@|\#|\.).+', elem)==None:
            elem = re.sub('[^(a-z|\')]', '', elem)
            if elem!='' and elem!='\'':
                split_words.append(elem)
    return split_words


In [19]:
def evaluate_tweet(tweet, lookup_dict, parse_funct):
    hit_count, score = 0, 0
    for word in parse_funct(tweet):
        if word in lookup_dict:
            hit_count += 1
            score += lookup_dict[word]
    if hit_count==0:
        score = np.nan
    else:
        score = score/hit_count
    return hit_count, score

In [37]:
hit_counts, scores = [], []
for i in tqdm(range(df.shape[0])):
    hit_count, score = evaluate_tweet(tweet=df['tweet_text'][i], lookup_dict=word_dict, parse_funct=get_words_advanced)
    hit_counts.append(hit_count)
    scores.append(score)

df = pd.DataFrame({'tweet_id':df['tweet_id'], 'hits':hit_counts, 'score': scores})


100%|██████████| 154621/154621 [00:06<00:00, 24536.82it/s]


In [38]:
df

,tweet_id,hits,score
0,1150558058994720776,1,0.200
1,1150558090044956673,4,0.625
2,1150558073771180032,1,0.200
3,1150558072911196160,1,0.800
4,1150558085716426752,2,0.500
...,...,...,...
154616,1150915001802248193,3,0.100
154617,1150914995367993344,2,0.500
154618,1150915001378410496,0,NaN
154619,1150915001479077889,10,0.350


In [39]:
df.to_csv('../data/features/indonesian_sentiment.csv', index=False)